In [1]:
!pip install pandas numpy requests

In [16]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO, StringIO
from google.colab import files  # Para facilitar o download de arquivos no Colab

# Função para carregar dados de uma URL ou caminho local
def carregar_dados(fonte, tipo='csv'):
    print(f"Carregando dados da fonte: {fonte}")

    try:
        # Carregar dados de URL
        if fonte.startswith('http'):
            response = requests.get(fonte)
            response.raise_for_status()  # Garantir que a requisição foi bem-sucedida
            if tipo == 'csv':
                return pd.read_csv(StringIO(response.text))
            elif tipo == 'excel':
                return pd.read_excel(BytesIO(response.content))
            elif tipo == 'geojson':
                return pd.read_json(response.text)
        else:
            # Carregar dados de caminho local
            if tipo == 'csv':
                return pd.read_csv(fonte)
            elif tipo == 'excel':
                return pd.read_excel(fonte)
            elif tipo == 'geojson':
                return pd.read_json(fonte)
    except Exception as e:
        print(f"Erro ao carregar os dados: {e}")
        return None

# Função para padronizar os nomes das colunas
def padronizar_colunas(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    return df

# Função para remover duplicatas
def remover_duplicatas(df):
    before = len(df)
    df = df.drop_duplicates()
    after = len(df)
    removed = before - after
    return df, removed

# Função para tratar valores nulos (remover ou preencher)
def tratar_nulos(df, metodo='drop', valor_preencher=0):
    before = len(df)
    if metodo == 'drop':
        df = df.dropna()
    elif metodo == 'fill':
        df = df.fillna(valor_preencher)
    after = len(df)
    removed = before - after
    return df, removed

# Função para normalizar categorias
def normalizar_categorias(df, coluna, substituicoes):
    df[coluna] = df[coluna].replace(substituicoes)
    return df

# Função para remover outliers usando z-score
def remover_outliers(df, colunas_numericas, z_thresh=3):
    removed = 0
    for col in colunas_numericas:
        if col in df.columns:
            z_scores = np.abs((df[col] - df[col].mean()) / df[col].std())
            before = len(df)
            df = df[z_scores < z_thresh]
            after = len(df)
            removed += before - after
    return df, removed

# Função principal para limpar e salvar dados
def limpar_e_salvar_dados(fonte, tipo='csv', col_cat=None, substituicoes_cat=None,
                           col_num=None, metodo_nulo='drop', valor_nulo=0):
    # Carregar dados
    df = carregar_dados(fonte, tipo)
    if df is None:
        return

    print(f"Total de registros carregados: {len(df)}")

    # Padronizar colunas
    df = padronizar_colunas(df)

    # Inicializar variáveis de remoções
    registros_removidos = []

    # Remover duplicatas
    df, duplicatas_removidas = remover_duplicatas(df)
    if duplicatas_removidas > 0:
        registros_removidos.append(f"Duplicatas removidas: {duplicatas_removidas} registros")

    # Tratar valores nulos
    df, nulos_removidos = tratar_nulos(df, metodo=metodo_nulo, valor_preencher=valor_nulo)
    if nulos_removidos > 0:
        registros_removidos.append(f"Registros com nulos removidos: {nulos_removidos} registros")

    # Normalizar categorias (se necessário)
    if col_cat and substituicoes_cat:
        df = normalizar_categorias(df, col_cat, substituicoes_cat)
        print(f"Categorias normalizadas na coluna: {col_cat}")

    # Remover outliers (se necessário)
    if col_num:
        df, outliers_removidos = remover_outliers(df, col_num)
        if outliers_removidos > 0:
            registros_removidos.append(f"Outliers removidos: {outliers_removidos} registros")

    # Exibir prévia do DataFrame limpo
    print("\nPrévia do DataFrame limpo:")
    print(df.head())

    # Salvar DataFrame limpo em CSV
    df.to_csv('/content/dados_limpos.csv', index=False)
    print("Arquivo CSV gerado: dados_limpos.csv")

    # Salvar DataFrame limpo como HTML interativo
    df.to_html('/content/dados_limpos.html', index=False, render_links=True)
    print("Arquivo HTML gerado: dados_limpos.html")

    # Salvar relatório de remoções
    if registros_removidos:
        with open('/content/remocoes.txt', 'w') as f:
            for item in registros_removidos:
                f.write(f"{item}\n")
        print("Arquivo de remoções gerado: remocoes.txt")
    else:
        with open('/content/remocoes.txt', 'w') as f:
            f.write("Sem remoção, dados limpos.\n")
        print("Arquivo de remoções gerado: remocoes.txt")

    # Baixar os arquivos gerados
    print("\nArquivos gerados com sucesso! Faça o download abaixo:")
    files.download('/content/dados_limpos.csv')
    files.download('/content/dados_limpos.html')
    files.download('/content/remocoes.txt')

    return df

# Exemplo de uso:
fonte = "https://raw.githubusercontent.com/josecacciatore89/acidentes_aereos/refs/heads/main/acidentes_aereos_dados_fornecidos.csv"
limpar_e_salvar_dados(fonte, tipo='csv')


Carregando dados da fonte: https://raw.githubusercontent.com/josecacciatore89/acidentes_aereos/refs/heads/main/acidentes_aereos_dados_fornecidos.csv
Total de registros carregados: 454

Prévia do DataFrame limpo:
    ano      data_do_acidente país_do_acidente       modelo_da_aeronave  \
0  1919   2 de agosto de 1919           Itália           Caproni Ca. 48   
1  1920        14 de dezembro       Inglaterra  Handley Page Type O-400   
2  1922    7 de abril de 1922           França         Havilland DH.18A   
3  1923    14 de maio de 1923           França      Farman F.60 Goliath   
4  1923  27 de agosto de 1923       Inglaterra      Farman F.60 Goliath   

  número_de_mortos  
0             13.5  
1              4.0  
2              7.0  
3              6.0  
4              1.0  
Arquivo CSV gerado: dados_limpos.csv
Arquivo HTML gerado: dados_limpos.html
Arquivo de remoções gerado: remocoes.txt

Arquivos gerados com sucesso! Faça o download abaixo:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,ano,data_do_acidente,país_do_acidente,modelo_da_aeronave,número_de_mortos
0,1919,2 de agosto de 1919,Itália,Caproni Ca. 48,13.5
1,1920,14 de dezembro,Inglaterra,Handley Page Type O-400,4.0
2,1922,7 de abril de 1922,França,Havilland DH.18A,7.0
3,1923,14 de maio de 1923,França,Farman F.60 Goliath,6.0
4,1923,27 de agosto de 1923,Inglaterra,Farman F.60 Goliath,1.0
...,...,...,...,...,...
447,2025,9 de março,Brasil (Santos),Cessna 150J,1
448,2025,12 de março,Brasil (Nova Andradina),Avião agrícola,1
449,2025,26 de fevereiro,Sudão,Antonov AN-12,46
452,2025,13 de março,EUA (Denver),Boeing 737 Next Generation (American Airlines),0
